## Cálculos de Pérdidas en Tuberias

<h5>Gustavo Raush, 2019</h5>

### Problemas simples de tuberias

Hacen referencias a tubos o tuberias donde la fricción del fluido con las paredes de las mismas esla única pérdida. El fluido es tratado como incompresible y las variables implicadas son: $Q$, $L$, $D$, $h_f$, $\nu$ y $\epsilon$. Generalmente, los datos conocidos son la longitud desarrollada $L$ , las porpiedades del fluidos como la viscosidad cinemática $\nu$ y la rugosidad de las paredes de las mismas $\epsilon$. 

Los problemas simples de tuberias se pueden dividir en tres tipos:

|Tipo|Dato|Incognita|
|----|----|---------|
|I|$Q$,$L$,$D$,$\nu$,$\epsilon$|$h_f$|
|II|$h_f$,$L$,$D$,$\nu$,$\epsilon$|$Q$|
|III|$h_f$,$Q$,$L$,$\nu$,$\epsilon$|$D$|

Para cada situación se ocupa la ecuación de Darcy-Weisbach, DW, la ecuación de la continuidad de la masa y el diagrama de Moody. Estas tres relaciones permiten encontrar la cantidad desconocida de la columna 3 de la tabla de arriba.

La Ecuación de DW, representa la relación de dos números adiemnsionales $f$, coeficiente de fricción y la relación de escala dimensional $L/D$ con la energía cinética que trae el del propio flujo $V^2/2g$, 

$$h_f = f \frac{L}{D} \frac{V^2}{2g}$$

A través del diagrama de Moody el coeficiente de fricción $f$ queda relacionado con el número de $\mathbf{Re}$ y parametrizado con la rugosidad adimensionalizada en el diámetro $\epsilon_r = \epsilon/D$ 

La mejor expresión que describe este conjunto de datos experimentales es la expresión de Colebrook,

$$\begin{array}{lcr}\frac{1}{\sqrt{f}} = - 2 \log_{10}\left( \frac{\epsilon}{3.7D} + \frac{2.51}{\mathbf{Re}\sqrt{f}}\right) & ; & 4000\le\mathbf{Re} \end{array}$$

Observamos que la misma tiene un caracter implícito en $f$.  Esto significa que no se puede resolver sin el auxilio de métodos numéricos. 

Pero existen relaciones explícitas para el calculo del coeficiente de fricción $f$ basado en una aproximación explícita sin cometer mayores errores al $1\%$. Esto tiene un rango de valides. 

|  | Restricciones |
|--|--|
|$f = \frac{1.325}{\left[ \ln \left( \frac{\epsilon}{3.7D} + \frac{5.74}{\mathbf{Re}^{0.9}}\right) \right]^2}$ | \begin{array}{ll} 10^{-6}\le\frac{\epsilon}{D}\le 10^{-2};\\ 5000\le\mathbf{Re}\le 10^8 \end{array}|

Antes de proceder con los ejemplos de solución se crean funciones de cálculo que facilitan y agilizan las ejemplificaciones. 

<h4>Definición de Funciones e Importación de Constantes</h4>

Las siguientes instrucciones ayudan a simplificar los calculo porque solamente se usaranc omo funciones de trabajo. De la misma manera importamos constantes  $\pi$ o la aceleración de la gravedad $g$.

In [78]:
from math import pi, sqrt, log10
from scipy.constants import g
def area(d):
    return pi / 4 * d**2
def ReQ(Q,D,nu):
    return 4 * Q / pi / D / nu
def ReV(V,D,nu):
    return V * D / nu
def velocidad(Q,D):
    return pi / 4 * D^2 * Q  
def DarcyWeisbach(f,L,D,V):
    return f * L / D * V**2 / 2 / g
def SwameeJainEq(D,eps,Re):
    return 0.25 / (log10((eps / D) / 3.7 + 5.74 / Re**0.9))**2

### Caso de Solución de Tipo I: Pérdida de Carga

Son conocidos $Q$, $D$, $L$, $\epsilon$ y el número de Reynolds expresados en términos del caudal $\mathbf{Re} = VD/\nu = 4Q/\pi D\nu$.
El objetivo es la solución directa de la ecuación de Darcy-Weisbach

> Ejemplo Tipo I: Determinar las pérdidas de cabeza piezométrica $h_f$ para el caso de un flujo de caudal $Q = 150\, L/s$ de agua $\nu = 10^{-6}\, m^2/s$ a través de una tubería de $L = 1500\, m$ de tubería de acero de $D = 250 \,mm$.
La rugosidad de una tubería de  acero tiene una rugosidad $\epsilon = 1.5\mu m$

In [79]:
Q = 0.15 # m^3/s
nu = 1.e-6 # m^2/s
D = 0.25 # m
L = 1500 # m
epsilon = 1.5e-6 # m
ReNum = ReQ(Q,D,nu)
print('Re = %2.0f' % ReNum)

Re = 763944


El valor es de un flujo totalmente turbulento, por lo que calculamos el valor de $f$ usando la ecuación de _Swamee–Jain_ 

In [17]:
f = SwameeJainEq(D,epsilon,ReNum)
print('f = %.4f' % f)

f = 0.0123


Similar valor lo podemos encontrar usando el Diagrama de Moody. Tambien resolviendo la ecuación de Colebrook usando herramientas numéricas. 

<h4>Pérdida de carga</h4>

La velocidad del flujo es

In [65]:
V = Q / area(D)
print('V = %.2f m/s' % V)

V = 3.06 m/s


La pérdida será,

In [71]:
hf = f * (L / D) * V**2 / 2 / g
print('hf = %.2f m' % hf)

hf = 35.07 m


El equivalente de la **pérdida de carga o cabeza** en términos de presión expresada en **Pa** es,

In [74]:
rho = 1000 # kg/m^3
phf = g * rho * hf
print('p = %.2f Pa' % phf)

p = 343959.16 Pa


<h3>Error entre la Ecuación de S-J y Colebrook</h3>

La rugosidad relativa $\epsilon/D$ es,

In [47]:
erNum = epsilon/D
print('er = %.1e' % erNum)

er = 6.0e-06


Las siguientes líneas nos permiten calcular el valor de $f$ directamente de una expresión modificada de la Ecuación de Colebrook que permite calcular el valor de $\sqrt{f}$

In [48]:
def fMoody(x,er,nre):
    return 1 + 2 * x * log10(er / 3.7 + 2.51 / nre / x)

In [49]:
fMoodyToSolve = lambda x : fMoody(x,er=erNum,nre=ReNum) 

In [50]:
from scipy.optimize import fsolve

In [52]:
fM = fsolve(fMoodyToSolve,0.01)[0]**2
print('f = %.4f' % fM)

f = 0.0123


El error comentido entre la Ecuación de SJ (aproximada) y la de Colebrook es,

In [62]:
Err = (f - fM)/fM * 100
print('Err = %.2f%%' % Err)

Err = -0.30%


El error es menor al 1%

### Caso de Solución de Tipo II: Cálculo de la Descarga Q

Son conocidos $h_f$, $D$, $L$, $\epsilon$ y el objetivo es el caudal $Q$

En este caso , la $V$ y $f$ son desconocido por lo que se tienen que usar las ecuacione de Darcy y Moody simultaneamente. Como se conoce $\epsilon/D$ se puede asumir un primer valor de $f$ con tal de conocer un valor aproximado de $V$ a través de Darcy. Esto mejora el valor de $\mathbf{Re}$ que permite mejorar el valor inicial de $f$ y repetir el cálculo a través de la iteración del mismo. 

> Ejemplo Tipo II: un sistema de tubería que transporta agua fluye a través de un tubo de acero con rugosidad $\epsilon = 0.01\, mm$ de diámetro $D = 100\, mm$ con una pérdida de carga $h_f = 6\, m$ y una longitud $L = 500\, m$


In [76]:
L = 500 # m
D = 0.1 # m
hf = 6 # m
eps = 0.01e-3 # m
rho = 1000 # kg/m^3
nu = 1e-6 # m^2/s

Darcy permite calcular la velocidad, $$h_f = f \frac{L}{D} \frac{V^2}{2g}$$ y la misma vale $$V = \sqrt{2g\frac{D}{f L} h_f}$$

Se adopta un valor de prueba $f = 0.02$ y se obtiene un valor de $V$

In [77]:
f = 0.02
V = sqrt(2 * g * D / f / L * hf)
print('V = %.2f m/s' % V)

V = 1.08 m/s


Correspondiente con un Reynolds, $\mathbf{Re} = VD/\nu$,

In [83]:
ReNum = ReV(D=D,V=V,nu=nu)
print('Re = %.0f' % ReNum)

Re = 271201


Flujo de régimen totalmente turbulento. La primera aproximación del valor del caudal $Q$,

In [84]:
Q = V * area(D)
print('Q = %.3f m^3/s' % Q)

Q = 0.053 m^3/s


Con el nuevo valor de $\mathbf{Re} = 271201$ y $\epsilon/D$ se mejora el $f$ usando Moody

In [86]:
f = SwameeJainEq(eps=eps,D=D,Re=ReNum)
print('f = %.3f' % f)

f = 0.015


Mejorando $V$ y $\mathbf{Re}$

In [87]:
V = sqrt(2 * g * D / f / L * hf)
print('V = %.2f m/s' % V)

V = 1.14 m/s


In [88]:
ReNum = ReV(D=D,V=V,nu=nu)
print('Re = %.0f' % ReNum)

Re = 285122


In [89]:
Q = V * area(D)
print('Q = %.3f m^3/s' % Q)

Q = 0.056 m^3/s


Este método se puede integrar dentro de un algoritmo de iteración.

In [158]:
def QFlow(hf,L,D,eps,nu,pre=4):
    err = 1 # precision de calculo
    fOld = 0.02 # valor inicial de semilla
    i = 0 # iteraciones
    while (abs(err) > 10**(-pre) and i < 10):
        V = sqrt(2 * g * D / fOld / L * hf)
        Re = ReV(V=V,D=D,nu=nu)
        fNew = SwameeJainEq(Re=Re,D=D,eps=eps)
        err = fNew - fOld
        fOld = fNew
        i += 1
        #print(abs(err),fOld,i)
    Q = V * area(D)
    return Q,i
        

La funcion ```QFlow``` devuelve el valor de $Q$ y el nñúmnero de iteraciones necesarias para llegar a una precisión de 3 decimales.

In [159]:
QFlow(hf=hf,L=L,D=D,eps=eps,nu=nu)

(0.05618173546046294, 3)

Con 2 iteraciones ya se consigue una aproximación de una milésima. 

In [160]:
Qaprox = QFlow(hf=hf,L=L,D=D,eps=eps,nu=nu,pre=5)[0]
print('Qaprox = %.4f m^3/s' % Qaprox)


Qaprox = 0.0562 m^3/s


De la misma manera vemos que para el caso de 5 decimales, solo basta un total de 4 iteraciones. 

<b>Conclusiones</b>: Se pueden conseguir resultados muy precisos con muy pocas iteraciones.

<h3>Solución "Exacta" de $Q$</h3>

Existe la posibilidad de obtener un nuevo nñumero adimensional $f\mathbf{Re}^2$ que al ser reemplazado en la ecuación de Colebrook permite ontener el valor exacto de $1/\sqrt{f}$. Este nuevo valor puede ser incluido en la expresión de $$Q = \frac{\pi}{2}  \sqrt{\frac{D^5 g h_f}{2 f L}}$$

Valor exacto de $Q$, con la precisión que permite la Ecuación de Colebrook.   
$$Q = -\frac{0.96476 D^{5/2} \sqrt{g} \sqrt{h_f} \log \left(\frac{1.79605 \sqrt{L}
   \nu }{D^{3/2} \sqrt{g} \sqrt{h_f}}+0.27027 \epsilon_r\right)}{\sqrt{L}}$$

In [111]:
from math import log
def QExact(hf,L,D,eps,nu):
    hL = hf
    Nu = nu
    er = eps/D
    C1 = -0.9647597718921425
    C2 = 0.27027027027027023
    C3 = 1.7960512242138307
    return (C1 * D**2.5 * sqrt(g) * sqrt(hL) * log(C2 * er + (C3*sqrt(L)*Nu) / 
                                                   (D**1.5 * sqrt(g) * sqrt(hL))))/sqrt(L)

In [121]:
Qexact = QExact(hf=hf,L=L,D=D,eps=eps,nu=nu)
print('Qexc = %.4f m^3/s' % Qexact)

Qexc = 0.0561 m^3/s


Error cometido entre $Q_{exact}$ y $Q_{aprox}$

In [115]:
Err = (Qaprox - Qexact)/Qexact * 100
print('Err = %.2f %%' % Err )

Err = 0.27 %


La expresión de $Q$, anteriormente presentada, puede ser evaluada con el siguiente algoritmo:

<!-- ~~puede quedar resumido en el siguiente procedimiento:~~ -->

>1. $C_1 = \frac{2 g h_f D^3}{L \nu^2}$ 
>2. $C_2 = -2 \log_{10}\left( \frac{\epsilon/D}{3.7} + \frac{2.51}{\sqrt{C_1}}\right)$
>3. $f = \frac{1}{C_2^2}$
>4. $Q = \frac{\pi}{2} \sqrt{g h_f D^5}{2 f L}$

Implementación del algoritmo:

In [116]:
def QexactAlg(hf,L,D,eps,nu):
    er = eps/D
    fR2 = (2 * D**3 * g * hf)/(L * nu**2)
    sqF = - 2 * log10(er / 3.7 + 2.51 / sqrt(fR2))
    f = 1 / sqF**2
    Q = pi / 2 * sqrt( g * hf * D**5 / 2 / f / L)
    return Q

In [120]:
Qexact2 = QexactAlg(hf=hf,L=L,D=D,eps=eps,nu=nu)
print('Qexc = %.4f m^3/s' % Qexact2)

Qexc = 0.0561 m^3/s


### Caso de Solución de Tipo III: Cálculo del Diámetro $D$

Son conocidos $h_f$, $Q$, $L$, $\epsilon$ y el objetivo es el dimensionado de la tubería  $D$

En este caso, hay tres incógnitas en las ecuacion de Darcy , la $f$, $V$ y $D$, dos incognitas en la ecuación de la continuidad $V$ y $D$ y tres en la ecuación del número de Reynolds $V$, $D$ y $\mathbf{Re}$. Por si fuera poco , la rugosidad relativa $\epsilon/D$ tampoco es conocida porque, aún conociendo el material $\epsilon$, se desconoce $D$.

<!-- son desconocido por lo que se tienen que usar las ecuacione de Darcy y Moody simultaneamente. Como se conoce $\epsilon/D$ se puede asumir un primer valor de $f$ con tal de conocer un valor aproximado de $V$ a través de Darcy. Esto mejora el valor de $\mathbf{Re}$ que permite mejorar el valor inicial de $f$ y repetir el cálculo a través de la iteración del mismo. 

> Ejemplo Tipo II: un sistema de tubería que transporta agua fluye a través de un tubo de acero con rugosidad $\epsilon = 0.01\, mm$ de diámetro $D = 100\, mm$ con una pérdida de carga $h_f = 6\, m$ y una longitud $L = 500\, m$-->


La Ecuación de Darcy para el caudal: $$h_f = f \frac{L}{D} \frac{Q^2}{2 g \left(D^2 \pi / 4\right)^2} = \frac{8 f L Q^2}{\pi ^2 D^5 g}$$

Despejando $$D^5 = \frac{8 L Q^2}{\pi^2 h_f g } f = C_1 f$$ Siendo $C_1 = \frac{8 L Q^2}{\pi^2 h_f g }$

Con ello se puede hacer una estimación del Reynolds, $$\mathbf{Re} = \frac{4 Q}{\pi \nu} \frac{1}{D} = \frac{C_2}{D}$$ Siendo $C_2 = \frac{4 Q}{\pi \nu}$

A partir de los valores de las constantes $C_1$ y $C_2$ ejecutar el algoritmo siguiente:


>1. Suponemos valor de $f = 0.02$
>2. Resolvemos $D^5 = C_1 f$
>3. Calculamos el valor de $\mathbf{Re} = \frac{C2}{D}$
>4. Encontrar la rugosidad relativa $\epsilon/D$
>5. Con $\mathbf{Re}$ y  $\epsilon/D$ , usar Moody para calcular $f_{new}$ nuevo
>6. Utilizando $f_{new}$ repetir calculo desde el paso (2)
>7. Detener el cálculo cuando $f$ ya no cambie. Se puede dar por bueno el valor de $D$

Aplicaremos el procemiento descrito al siguiente caso práctico:

> Ejemplo Tipo III: determine la medida de un tubo de hierro colado $\epsilon = 0.045\,mm$ necesario para la conducción de  $4\,m³/s$ de aceite $\nu = 0.01 m²/s$. El sistema tienen una longitud de 100 m y se desea una pérdida de carga de 0.1 m/m.
<!-- un sistema de tubería que transporta agua fluye a través de un tubo de acero con rugosidad $\epsilon = 0.01\, mm$ de diámetro $D = 100\, mm$ con una pérdida de carga $h_f = 6\, m$ y una longitud $L = 500\, m$-->


In [204]:
from math import pow # importación de la función para la raiz quinta

In [205]:
def DDesign(Q,hf,L,eps,nu):
    fOld = 0.02
    C1 = (8 * L * Q**2)/(pi**2 * g* hf )
    C2 = (4 * Q)/(pi * nu)
    i = 0
    err = 1
    while (abs(err > 0.001) and i < 10):
        D5 = C1 * fOld
        Dnew = pow(D5,1/5)
        ReNew = C2 / D
        errNew = eps/D
        fNew = SwameeJainEq(Re=ReNew,D=Dnew,eps=eps)
        err = fNew - fOld
        fOld = fNew
        i += 1
    return Dnew,i
        

In [202]:
Q = 4 # m³/s
L = 100 # m
hf = 0.1 * L # m
nu = 0.01 # m²/s
eps = 0.045e-3 # m

In [203]:
DDesign(Q=Q,hf=hf,L=L,eps=45e-6,nu=nu)

(0.9235819053267553, 2)

La solución se consigue con muy pocas iteraciones. 


La solución se puede dar como $D = 0.923 \,mm$